# Tema 15: El TAD de las colas

[José A. Alonso](https://www.cs.us.es/~jalonso)  
[Departamento de Ciencias de la Computación e I.A.](https://www.cs.us.es)  
[Universidad de Sevilla](http://www.us.es)  
Sevilla, 11 de agosto de 2019

> __Notas:__ 
+ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-15.ipynb).
+ Se desactiva el [corrector estilo de Haskell](https://github.com/gibiansky/IHaskell/wiki#opt-no-lint).

In [1]:
:opt no-lint

Especificación del TAD de las colas
===================================

Signatura del TAD de las colas
------------------------------

**Descripción informal de las colas**

+ Una *cola* es una estructura de datos, caracterizada por ser una secuencia de
  elementos en la que la operación de inserción se realiza por un extremo (el
  posterior o final) y la operación de extracción por el otro (el anterior o
  frente).

+ Las colas también se llaman estructuras FIFO (del inglés First In First Out),
  debido a que el primer elemento en entrar será también el primero en salir.

+ Analogía con las colas del cine.

**Signatura del TAD colas**

+ Signatura:

```sesion
vacia   :: Cola a
inserta :: a -> Cola a -> Cola a
primero :: Cola a -> a
resto   :: Cola a -> Cola a
esVacia :: Cola a -> Bool
valida  :: Cola a -> Bool
```

+ Descripción de las operaciones:
    + `vacia` es la cola vacía.
    + `(inserta x c)` es la cola obtenida añadiendo `x` al final de `c`.
    + `(primero c)` es el primero de la cola `c`.
    + `(resto c)` es la cola obtenida eliminando el primero de `c`.
    + `(esVacia c)` se verifica si `c` es la cola vacía.
    + `(valida c)` se verifica si `c` representa una cola válida.

Propiedades del TAD de las colas
--------------------------------

**Propiedades del TAD de las colas**

+ `primero (inserta x vacia) == x`

+ Si `c` es una cola no vacía, entonces `primero (inserta x c) == primero c`,

+ `resto (inserta x vacia) == vacia`

+ Si `c` es una cola no vacía, entonces `resto (inserta x c) == inserta x
  (resto c)`

+ `esVacia vacia`

+ `not (esVacia (inserta x c))`

Implementaciones del TAD de las colas
=====================================

Implementación de las colas mediante listas
-------------------------------------------

Está en el fichero [ColaConListas.hs](codigos/ColaConListas.hs).

In [2]:
module ColaConListas 
    (Cola,
     vacia,   -- Cola a
     inserta, -- a -> Cola a -> Cola a
     primero, -- Cola a -> a
     resto,   -- Cola a -> Cola a
     esVacia, -- Cola a -> Bool
     valida   -- Cola a -> Bool
    ) where

-- Colas como listas:
newtype Cola a = C [a]
    deriving (Show, Eq)

-- Ejemplo de cola
--    ghci> c1
--    C [10,9,8,7,6,5,4,3,2,1]
c1 = foldr inserta vacia [1..10]

-- vacia es la cola vacía. Por ejemplo,
--    ghci> vacia
--    C []
vacia :: Cola a
vacia = C []

-- (inserta x c) es la cola obtenida añadiendo x al final de la cola
-- c. Por ejemplo,
--    inserta 12 c1  ==  C [10,9,8,7,6,5,4,3,2,1,12]
inserta :: a -> Cola a -> Cola a
inserta x (C c) = C (c ++ [x])

-- Nota: La operación inserta usa O(n) pasos.

-- (primero c) es el primer elemento de la cola c. Por ejemplo,
--    primero c1  ==  10
primero :: Cola a -> a
primero (C (x:_)) = x
primero (C [])    = error "primero: cola vacia"

-- (resto c) es la cola obtenida eliminando el primer elemento de la
-- cola c. Por ejemplo,
--    resto c1  ==  C [9,8,7,6,5,4,3,2,1]
resto :: Cola a -> Cola a
resto (C (_:xs)) = C xs
resto (C [])     = error "resto: cola vacia"

-- (esVacia c) se verifica si c es la cola vacía. Por ejemplo,
--    esVacia c1     ==  False
--    esVacia vacia  ==  True
esVacia :: Cola a -> Bool
esVacia (C xs)  = null xs

-- (valida c) se verifica si c representa una cola válida. Con esta
-- representación, todas las colas son válidas.
valida :: Cola a -> Bool
valida c = True

+ Ejemplos

In [3]:
c1 = foldr inserta vacia [1..10]

In [4]:
c1

C [10,9,8,7,6,5,4,3,2,1]

In [5]:
vacia

C []

In [6]:
inserta 12 c1

C [10,9,8,7,6,5,4,3,2,1,12]

In [7]:
primero c1

10

In [8]:
resto c1

C [9,8,7,6,5,4,3,2,1]

In [9]:
esVacia c1

False

In [10]:
esVacia vacia

True

+ Se borra la primera implementación

In [11]:
:m - ColaConListas

In [12]:
esVacia vacia

: 

Implementación de las colas mediante pares de listas
----------------------------------------------------

**Las colas como pares de listas**

+ En esta implementación, una cola `c` se representa mediante un par de listas
  `(xs,ys)` de modo que los elementos de `c` son, en ese orden, los elementos
  de la lista `xs++(reverse ys)`.

+ Al dividir la lista en dos parte e invertir la segunda de ellas, esperamos
  hacer más eficiente las operaciones sobre las colas.

+ Impondremos también una restricción adicional sobre la representación: las
  colas serán representadas mediante pares `(xs,ys)` tales que si `xs` es
  vacía, entonces `ys` será también vacía.

+ Esta restricción ha de mantenerse por las operaciones que crean colas.

**Implementación de las colas como pares de listas**

Está en el fichero [ColaConDosListas.hs](codigos/ColaConDosListas.hs).

In [13]:
module ColaConDosListas
    (Cola,
     vacia,   -- Cola a
     inserta, -- a -> Cola a -> Cola a
     primero, -- Cola a -> a
     resto,   -- Cola a -> Cola a
     esVacia, -- Cola a -> Bool
     valida   -- Cola a -> Bool
    ) where

-- Las colas como pares listas.
newtype Cola a = C ([a],[a])
    -- deriving Show

-- Procedimiento de escritura de colas como pares de listas.
instance (Show a) => Show (Cola a) where
  showsPrec _ (C (xs,ys)) cad =
    showString "C " (showList (xs ++ reverse ys) cad)

-- Ejemplo de cola: c1 es la cola obtenida añadiéndole a la cola 
-- vacía los números del 1 al 10. Por ejemplo,
--    ghci> c1
--    C [10,9,8,7,6,5,4,3,2,1]
c1 :: Cola Int
c1 = foldr inserta vacia [1..10]

-- vacia es la cola vacía. Por ejemplo,
--    ghci> vacia
--    C []
vacia :: Cola a
vacia  = C ([],[])

-- (inserta x c) es la cola obtenida añadiendo x al final de la cola
-- c. Por ejemplo,
--    inserta 12 c1  ==  C [10,9,8,7,6,5,4,3,2,1,12]
inserta :: a -> Cola a -> Cola a
inserta y (C (xs,ys)) = C (normaliza (xs,y:ys))

-- (normaliza p) es la cola obtenida al normalizar el par de listas
-- p. Por ejemplo,  
--    normaliza ([],[2,5,3])   ==  ([3,5,2],[])
--    normaliza ([4],[2,5,3])  ==  ([4],[2,5,3])
normaliza :: ([a],[a]) -> ([a],[a])
normaliza ([], ys) = (reverse ys, [])
normaliza p        = p

-- (primero c) es el primer elemento de la cola c. Por ejemplo,
--    primero c1  ==  10
primero  :: Cola a -> a
primero (C (x:_,_)) = x
primero _           = error "primero: cola vacia"

-- (resto c) es la cola obtenida eliminando el primer elemento de la
-- cola c. Por ejemplo,
--    resto c1  ==  C [9,8,7,6,5,4,3,2,1]
resto  :: Cola a -> Cola a
resto (C ([],[]))   = error "resto: cola vacia"
resto (C (_:xs,ys)) = C (normaliza (xs,ys))

-- (esVacia c) se verifica si c es la cola vacía. Por ejemplo,
--    esVacia c1     ==  False
--    esVacia vacia  ==  True
esVacia :: Cola a -> Bool
esVacia (C (xs,_)) = null xs

-- (valida c) se verifica si la cola c es válida; es decir, si
-- su primer elemento es vacío entonces también lo es el segundo. Por
-- ejemplo, 
--    valida (C ([2],[5]))  ==  True
--    valida (C ([2],[]))   ==  True
--    valida (C ([],[5]))   ==  False
valida:: Cola a -> Bool
valida (C (xs,ys)) = not (null xs) || null ys

-- ---------------------------------------------------------------------
-- Igualdad de colas                                                  --
-- ---------------------------------------------------------------------

-- (elementos c) es la lista de los elementos de la cola c en el orden de
-- la cola. Por ejemplo, 
--    elementos (C ([3,2],[5,4,7]))  ==  [3,2,7,4,5]
elementos :: Cola a -> [a]
elementos (C (xs,ys)) = xs ++ reverse ys

-- (igualColas c1 c2) se verifica si las colas c1 y c2 son iguales. Por
-- ejemplo, 
--    igualColas (C ([3,2],[5,4,7])) (C ([3],[5,4,7,2]))   ==  True
--    igualColas (C ([3,2],[5,4,7])) (C ([],[5,4,7,2,3]))  ==  False
igualColas :: Eq a => Cola a -> Cola a -> Bool
igualColas c c' = 
  valida c && valida c' && elementos c == elementos c'

instance (Eq a) => Eq (Cola a) where
  (==) = igualColas

+ Ejemplos

In [14]:
c1 = foldr inserta vacia [1..10]

In [15]:
c1

C [10,9,8,7,6,5,4,3,2,1]

In [16]:
vacia

C []

In [17]:
inserta 12 c1

C [10,9,8,7,6,5,4,3,2,1,12]

In [18]:
primero c1  

10

In [19]:
resto c1

C [9,8,7,6,5,4,3,2,1]

In [20]:
esVacia c1

False

In [21]:
esVacia vacia

True

+ Se borra la 2ª implementación

In [22]:
:m - ColaConDosListas

Comprobación de las implementaciones con QuickCheck
===================================================

In [23]:
{-# LANGUAGE FlexibleInstances #-}

module ColaPropiedades where

-- Hay que elegir una implementación del TAD colas:
import ColaConListas
-- import ColaConDosListas

import Test.QuickCheck

-- ---------------------------------------------------------------------
-- Generador de colas                                          --
-- ---------------------------------------------------------------------

-- genCola es un generador de colas de enteros. Por ejemplo,
--    ghci> sample genCola
--    C ([],[])
--    C ([],[])
--    C ([],[])
--    C ([],[])
--    C ([7,8,4,3,7],[5,3,3])
--    C ([],[])
--    C ([1],[13])
--    C ([18,28],[12,21,28,28,3,18,14])
--    C ([47],[64,45,7])
--    C ([8],[])
--    C ([42,112,178,175,107],[])
genCola :: Gen (Cola Int)
genCola = frequency [(1, return vacia),
                     (30, do n <- choose (10,100)
                             xs <- vectorOf n arbitrary
                             return (creaCola xs))]
          where creaCola = foldr inserta vacia

-- El tipo pila es una instancia del arbitrario.
instance Arbitrary (Cola Int) where
  arbitrary = genCola

-- Propiedad. Todo los elementos generados por genCola son colas
-- válidas. 
prop_genCola_correcto :: Cola Int -> Bool
prop_genCola_correcto c = valida c

-- Comprobación.
--    ghci> quickCheck prop_genCola_correcto
--    +++ OK, passed 100 tests.

-- ---------------------------------------------------------------------
-- Propiedades
-- ---------------------------------------------------------------------

-- Propiedad. El primero de la cola obtenida añadiendo x a la cola vacía
-- es x. 
prop_primero_inserta_vacia :: Int -> Bool
prop_primero_inserta_vacia x = 
  primero (inserta x vacia) == x

-- Comprobación.
--    > quickCheck prop_primero_inserta_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. Si una cola no está vacía, su primer elemento no varía al
-- añadirle un elemento. 
prop_primero_inserta_no_vacia :: Cola Int -> Int -> Int -> Bool
prop_primero_inserta_no_vacia c x y =
  primero (inserta x c') == primero c'
  where c' = inserta y vacia

-- Comprobación.
--    > quickCheck prop_primero_inserta_no_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. El resto de la cola obtenida insertando un elemento en la
-- cola vacía es la cola vacía.  
prop_resto_inserta_vacia :: Int -> Bool
prop_resto_inserta_vacia x = 
  resto (inserta x vacia) == vacia

-- Comprobación.
--    > quickCheck prop_resto_inserta_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. Las operaciones de inserta y resto conmutan.
prop_resto_inserta_en_no_vacia :: Cola Int -> Int -> Int -> Bool
prop_resto_inserta_en_no_vacia c x y =
  resto (inserta x c') == inserta x (resto c')
  where c' = inserta y c

-- Comprobación.
--    > quickCheck prop_resto_inserta_en_no_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. vacia es una cola vacía.
prop_vacia_es_vacia :: Bool
prop_vacia_es_vacia = 
  esVacia vacia

-- Comprobación.
--    > quickCheck prop_vacia_es_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. La cola obtenida insertando un elemento no es vacía.
prop_inserta_no_es_vacia :: Int -> Cola Int -> Bool
prop_inserta_no_es_vacia x c = 
  not (esVacia (inserta x c))

-- Comprobación
--    > quickCheck prop_inserta_no_es_vacia
--    +++ OK, passed 100 tests.

-- ---------------------------------------------------------------------
-- Propiedades de la normalización                                    --
-- ---------------------------------------------------------------------

-- Propiedad. La cola vacía es válida.
prop_valida_vacia :: Bool
prop_valida_vacia = valida vacia

-- Comprobación
--    ghci> quickCheck prop_valida_vacia
--    +++ OK, passed 100 tests.

-- Propiedad. Al añadirle un elemento a una cola válida se obtiene otra
-- cola válida. 
prop_valida_inserta :: Cola Int -> Int -> Property
prop_valida_inserta c x =
  valida c ==> valida (inserta x c)

-- Comprobación.
--    ghci> quickCheck prop_valida_inserta
--    +++ OK, passed 100 tests.

-- Propiedad. El resto de una cola válida y no vacía es una cola válida. 
prop_valida_resto :: Cola Int -> Property
prop_valida_resto c =
  valida c && not (esVacia c) ==> valida (resto c)

-- Comprobación
--    ghci> quickCheck prop_valida_resto
--    +++ OK, passed 100 tests.

Comprobación de las propiedades
-------------------------------

In [24]:
import Test.QuickCheck
quickCheck prop_primero_inserta_vacia
quickCheck prop_primero_inserta_no_vacia
quickCheck prop_resto_inserta_vacia
quickCheck prop_resto_inserta_en_no_vacia
quickCheck prop_vacia_es_vacia
quickCheck prop_inserta_no_es_vacia
quickCheck prop_valida_vacia
quickCheck prop_valida_inserta
quickCheck prop_valida_resto

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests.

+++ OK, passed 1 test.

+++ OK, passed 100 tests.

+++ OK, passed 1 test.

+++ OK, passed 100 tests.

+++ OK, passed 100 tests; 3 discarded.

> **Nota** Se borran los ficheros de los módulos usados

In [25]:
:! rm -f *.hs *.hi *.o *.dyn_*

Referencias
===========

+ F. Rabhi y G. Lapalme
  [Algorithms: A functional programming approach](http://bit.ly/1mWBqiI)
    + Cap. 5.3. Queues.
+ WikiBooks, [Data structures primer](http://bit.ly/1mWAQRY).